In [13]:
from imagebind_dataloader import ImageText_DataLoader, VideoText_DataLoader
import torch
from torch import nn
from config import EPOCHS, IMAGE_TRANSFORM, VIDEO_TRANSFORM, DEVICE
from torch.utils.data import DataLoader
from model import ImageBindModel, ModalityType

import torch.nn.functional as F
from tqdm import tqdm


In [9]:

image_ds = ImageText_DataLoader(image_paths = "Data/Image_Data", transform = IMAGE_TRANSFORM)
image_dl = DataLoader(image_ds, batch_size=2, shuffle=True)

video_ds = VideoText_DataLoader(video_paths = "Data/Video_Data", transform = VIDEO_TRANSFORM)
video_dl = DataLoader(video_ds, batch_size=2, shuffle=True)

model = ImageBindModel()

In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
EPOCHS = 2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(DEVICE)

# Training Loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    step = 0

    pbar = tqdm(zip(image_dl, video_dl), desc=f"Epoch {epoch+1}/{EPOCHS}")

    for image_batch, video_batch in pbar:
        # Unpack image-text batch
        image, text = image_batch
        image, text = image.to(DEVICE), text.to(DEVICE)

        inputs = {
            ModalityType.VISION: image,
            ModalityType.TEXT: text
        }

        embeddings = model(inputs)

        # emb_image = F.normalize(embeddings[ModalityType.VISION], dim=-1)
        # emb_text = F.normalize(embeddings[ModalityType.TEXT], dim=-1)

        # Compute cosine similarity logits
        logits = embeddings['vision'] @ embeddings['text']

        # InfoNCE Loss
        labels = torch.arange(logits.size(0), device=DEVICE)
        loss_i2t = F.cross_entropy(logits, labels)
        loss_t2i = F.cross_entropy(logits.T, labels)
        loss = (loss_i2t + loss_t2i) / 2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print(total_loss)
        step += 1
        pbar.set_postfix({"Loss": total_loss / step})

    avg_loss = total_loss / step
    print(f"[Epoch {epoch+1}] Avg InfoNCE Loss: {avg_loss:.4f}")

Epoch 1/2: 0it [00:01, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x768 and 2x768)